In [53]:
from keras.applications.inception_v3 import InceptionV3,preprocess_input,decode_predictions
from keras.preprocessing import image
import numpy as np
from keras.layers import Dense, GlobalAveragePooling2D,Dropout,Input
from keras.models import Sequential,Model
from keras import backend as K
from IPython.display import display
import numpy as np
import pandas as pd
import os
from sklearn.datasets import load_files
from keras.utils import np_utils
import keras
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Flatten, BatchNormalization, Activation, Dropout
from keras.callbacks import ModelCheckpoint, TensorBoard

In [54]:
#Loading model
base_model  = InceptionV3(weights = 'imagenet', include_top=False)
print('loaded model')

loaded model


In [55]:
data_gen_args = dict(preprocessing_function=preprocess_input)

train_datagen = image.ImageDataGenerator(**data_gen_args)
test_datagen = image.ImageDataGenerator(**data_gen_args)

In [56]:
train_generator = train_datagen.flow_from_directory("D:\\ADI\\SkiSyAI\\DataSet\\NEW\\train",
                                                    target_size=(299,299),batch_size=32)

valid_generator = test_datagen.flow_from_directory("D:\\ADI\\SkiSyAI\\DataSet\\NEW\\test",
                                                     target_size=(299,299),batch_size=32)

Found 1844 images belonging to 4 classes.
Found 434 images belonging to 4 classes.


In [57]:
#building model
last = base_model.output
x = GlobalAveragePooling2D()(last)
x = Dense(512, activation='relu')(x)
preds = Dense(4,activation='softmax')(x)

model = Model(input=base_model.input,output=preds)

from keras.callbacks import ModelCheckpoint,EarlyStopping
# Save the model with best weights
checkpointer = ModelCheckpoint('saved_models/new_data.hdf5', verbose=1,save_best_only=True)

d:\softwares\anaconda\envs\skisyai\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  import sys


In [58]:
for layer in base_model.layers:
    layer.trainable = False

In [59]:
model.compile(optimizer='adam', loss = 'categorical_crossentropy',metrics=['accuracy'])


In [61]:
history_transfer = model.fit_generator(train_generator, steps_per_epoch = 5,validation_data=valid_generator,validation_steps=4, epochs=25,verbose=1,callbacks=[checkpointer])

Epoch 1/25
5/5 [==============================] - 6s 1s/step - loss: 0.6679 - acc: 0.7812 - val_loss: 0.8845 - val_acc: 0.6406

Epoch 00001: val_loss improved from 0.95537 to 0.88450, saving model to saved_models/new_data.hdf5
Epoch 2/25
5/5 [==============================] - 5s 903ms/step - loss: 0.6852 - acc: 0.7250 - val_loss: 0.9984 - val_acc: 0.5625

Epoch 00002: val_loss did not improve from 0.88450
Epoch 3/25
5/5 [==============================] - 5s 925ms/step - loss: 0.6780 - acc: 0.7562 - val_loss: 0.8397 - val_acc: 0.6404

Epoch 00003: val_loss improved from 0.88450 to 0.83975, saving model to saved_models/new_data.hdf5
Epoch 4/25
5/5 [==============================] - 4s 761ms/step - loss: 0.6701 - acc: 0.7625 - val_loss: 0.8256 - val_acc: 0.6094

Epoch 00004: val_loss improved from 0.83975 to 0.82556, saving model to saved_models/new_data.hdf5
Epoch 5/25
5/5 [==============================] - 3s 681ms/step - loss: 0.4846 - acc: 0.8438 - val_loss: 0.7704 - val_acc: 0.6641



OSError: image file is truncated (0 bytes not processed)

In [62]:
import matplotlib
from keras.preprocessing.image import img_to_array,load_img
import matplotlib.pyplot as plt
import cv2
%matplotlib inline

def pred(img_path):    
    img = load_img(img_path,target_size = (299,299)) #Load the image and set the target size to the size of input of our model
    x = img_to_array(img) #Convert the image to array
    x = np.expand_dims(x,axis=0) #Convert the array to the form (1,x,y,z) 
    x = preprocess_input(x) # Use the preprocess input function o subtract the mean of all the images
    p = np.argmax(model.predict(x)) # Store the argmax of the predictions
    print(p)

def plot_training(history):
    acc = history.history['acc'] 
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs  = range(len(acc))
    
    plt.plot(epochs,acc,'b')
    plt.plot(epochs,val_acc,'r')
    plt.title("Training and validation accuracy")
    
    plt.figure()
    plt.plot(epochs,loss,'b')
    plt.plot(epochs,val_loss,'r')
    plt.title("Training and validation loss")
    
    plt.show()

In [63]:
plot_training(history_transfer)

NameError: name 'history_transfer' is not defined

## GRAPHS AIN'T GOOD :/

### Testing

In [64]:
pred('D:\ADI\SkiSyAI\DataSet\Tinea Corposis\80.Tinea-corporis-l.jpg')

3


In [65]:
model.outputs

[<tf.Tensor 'dense_4/Softmax:0' shape=(?, 4) dtype=float32>]

In [66]:
labels = train_generator.class_indices

In [67]:
labels

{'Acne': 0, 'Alopecia': 1, 'Normal Skin': 2, 'Tinea': 3}

## App Section

In [68]:
from keras.models import load_model
from keras.preprocessing.image import img_to_array,load_img
import matplotlib.pyplot as plt
import cv2

def pred(img_path):    
    img = load_img(img_path,target_size = (299,299)) #Load the image and set the target size to the size of input of our model
    x = img_to_array(img) #Convert the image to array
    x = np.expand_dims(x,axis=0) #Convert the array to the form (1,x,y,z) 
    x = preprocess_input(x) # Use the preprocess input function o subtract the mean of all the images
    p = np.argmax(model.predict(x)) # Store the argmax of the predictions
    Labels={'Acne': 0, 'Alopecia': 1, 'Normal Skin': 2, 'Tinea': 3}
    rev_label = {val:key for key,val in Labels.items()}
    ##PS : MODEL GIVES INDEX WE NEED TO RETURN LABEL
    return rev_label[p]


model = load_model('saved_models/new_data.hdf5')

In [70]:
pred("D:\ADI\SkiSyAI\DataSet\Tinea Corposis\80.Tinea-corporis-l.jpg")

'Tinea'

In [71]:
pred("C:\\Users\\Aditya Srivastava\\Pictures\\Camera Roll\\adi.jpg")

'Tinea'

In [72]:
pred("D:\\ADI\\SkiSyAI\\DataSet\\NEW\\train\\Normal Skin\\back male27.jpg")

'Normal Skin'

## :)

In [73]:
pred("C:\\Users\\Aditya Srivastava\\Desktop\\WhatsApp Image 2019-08-04 at 6.22.42 PM.jpeg")

'Normal Skin'

In [74]:
pred("C:\\Users\\Aditya Srivastava\\Desktop\\WhatsApp Image 2019-08-04 at 6.22.36 PM.jpeg")

'Tinea'

from os import listdir
from PIL import Image
    
for filename in listdir('DataSet/NEW/test/Normal Skin/'):
        try:
            img = Image.open('DataSet/NEW/test/Normal Skin/'+filename) # open the image file
            img.verify() 
            print(img.shape)
            # verify that it is, in fact an image
        except (IOError, SyntaxError) as e:
              print('Bad file:', filename) # print